<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png" width="40px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Prompt Engineering for Quality Decisions</span>
</div>

Decision Intelligence applied in this module:  
* Optimizing prompts for logic, reasoning and decisions
* Examples of applying decision intelligence techniques in prompts and their output effect 
* Prompt techniques such as Chain of Thought to improve the quality of the decision reasoning and outcome 


In this module, the Semantic Kernel ability to chat completion experience will be used to optimize system prompts (personas) and instructive prompts. No new specific Semantic Kernel functionality will be introduced. This module will focus on introducing decision intelligence prompting techniques.  

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [21]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.9"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.9"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
var azureOpenAIReasoningEndpoint = config["AzureOpenAI:ReasoningEndpoint"];
var azureOpenAIReasoningAPIKey = config["AzureOpenAI:ReasoningAPIKey"];
var azureOpenAIReasoningModelDeploymentName = config["AzureOpenAI:ReasoningModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.9 Microsoft.Extensions.Configuration.Json, 9.0.9

In [22]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.9"
#r "nuget: Microsoft.SemanticKernel, 1.65.0"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 9.0.9 Microsoft.SemanticKernel, 1.65.0

Using Azure OpenAI Service


### Step 2 - Chain of Thought Reasoning for Decision Intelligence 

> 📜 **_"The happiness of your life depends upon the quality of your thoughts."_**
>
> -- <cite>Marcus Aurelius (Roman emperor, philosopher)</cite>  

One of the most effective techniques to improve logic, reasoning and overall decision-making for most LLMs is to use "Chain of Thought" (CoT) techniques. The "Chain of Thought" approach helps by breaking down complex tasks into smaller steps, making it easier to solve problems without missing important details. It improves accuracy because each step is made clear to the LLM and helps avoid mistakes. This method also makes the thought process easier to understand and explain, allowing for easier corrections when something goes wrong. The approach is useful for handling difficult concepts and multi-step problems like math or logic, making things clearer and more manageable.

The name of the "Chain of Thought" technique has been recently popularized by Generative AI. However, this technique is not new nor is it specific only to Generative AI models. Breaking down complex tasks into simpler more approachable steps and organizing thoughts has been used by decision-makers, professional services organizations and management consulting companies for quite some time. Prior to Generative AI, the concepts have been known as: "structured thinking", "MECE framework" (Mutually Exclusive, Collectively Exhaustive), and "hypothesis-driven problem-solving".  

First, approach the problem with a simple Decision prompt technique. The system decisoin prompt will be set to basic & clear decision assistant persona instructions. While this decision prompt is simple, it is still quite effective to instruct the LLM model on how to approach the decision problem. 

In [23]:
// Set the system prompt to behave like a decision intelligence assistant (persona)
var systemDecisionPrompt = """
You are a decision intelligence assistant. 
Provide structured, logical, and comprehensive advice.

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 

Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";

// Simple instruction prompt to plan retirement
var puzzlePrompt = """
A farmer is on one side of a river with a wolf, a goat, and a cabbage. 
When he is crossing the river in a boat, he can only take one item with him at a time. 
The wolf will eat the goat if left alone together, and the goat will eat the cabbage if left alone together. 
How can the farmer transport the goat across the river without it being eaten?
""";

// Build Chat History with system prompt and the puzzle prompt
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemDecisionPrompt);
chatHistory.AddUserMessage(puzzlePrompt);

// Create a new OpenAI prompt execution settings object
PromptExecutionSettings promptExecutionSettings = new();

if (useAzureOpenAI)
{
    // Create a new Azure OpenAI Prompt Execution settings object
    #pragma warning disable SKEXP0010
    promptExecutionSettings = new AzureOpenAIPromptExecutionSettings { 
        SetNewMaxCompletionTokensEnabled = true,
        MaxTokens = 1500,
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0, 
        PresencePenalty = 0.0
        };
}
else
{
    // Create a new OpenAI Prompt Execution settings object
    promptExecutionSettings = new OpenAIPromptExecutionSettings { 
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0, 
        PresencePenalty = 0.0
        };
}

// Execute the chat completion request using the prompt and settings defined above
var decisionResponse = await chatCompletionService.GetChatMessageContentAsync(chatHistory, promptExecutionSettings);
// Display the result of the chat completion request as Markdown
decisionResponse.Content.DisplayAs("text/markdown");
("---").DisplayAs("text/markdown"); // Break of the ooutput section

| Step | Left Bank Contents                 | Right Bank Contents | Action |
|------|-------------------------------------|----------------------|--------|
| 1    | Farmer, Wolf, Goat, Cabbage         | -                    | Farmer takes Goat to right bank |
| 2    | Farmer, Wolf, Cabbage               | Goat                 | Farmer returns alone to left bank |
| 3    | Farmer, Wolf, Cabbage               | Goat                 | Farmer takes Wolf to right bank |
| 4    | Goat, Farmer                        | Wolf                 | Farmer brings Goat back to left bank |
| 5    | Farmer, Goat, Cabbage               | Wolf                 | Farmer takes Cabbage to right bank |
| 6    | Goat, Farmer                        | Wolf, Cabbage        | Farmer returns alone to left bank |
| 7    | Farmer, Goat                        | Wolf, Cabbage        | Farmer takes Goat to right bank |



---

Let's approach the same problem with a much more sophisticated "Chain of Thought" (CoT) system prompt to break the problem down and think about it more in depth. Notice that the prompt below is much more detailed in how the "decision intelligence assistant" should approach the problem, highlighting things it should or should not consider and the way it should arrive at the final answer. 

Chain of Thought (CoT) prompt instructions will vary depending on how the problem needs to be approached. It can include very specific decision framework instructions, systematic decision heuristics or management consulting best practices. In general, these detailed prompts result in the LLM providing more detail and usually better outcomes. However, there are a few drawbacks to overusing Chain of Thought (CoT):
* A very long and detailed Chain of Thought (CoT) can confuse the LLM model, especially if the LLM model is smaller (i.e. GPT-4o-mini or Phi-4 etc.)
* GenAI LLM models can hallucinate not just on output of answers. They can also hallucinate the Chain of Thought (CoT) they are describing in the final answer! This is a very sneaky way of potentially providing a confident answer paired with a confident approach (Chain of Thought) that the LLM may not even be using! 

In [24]:
// Set the system prompt to behave like a decision intelligence assistant (persona)
var systemPromptChainOfThought = """
You are a Decision Intelligence assistant designed to think through problems step-by-step using Chain-of-Thought (CoT) prompting. 
Before providing any answer, you must: 

1) Understand the Problem: Carefully read and understand the user's question or request. 

2) Break Down the Reasoning Process: Outline the steps required to solve the problem or respond to the request logically and sequentially. Think aloud and describe each step in detail. 
Explain Each Step: Provide reasoning or calculations for each step, explaining how you arrive at each part of your answer. 
Provide structured, logical, and comprehensive advice. 

3) Review the Thought Process: Double-check the reasoning for errors or gaps before finalizing your response. 
Always aim to make your thought process transparent and logical.

4) Arrive at the Final Answer: Only after completing all steps, provide the detailed final answer or detailed solution. 

5) Communicate the final decision details using the Minto Pyramid Principle. 

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 

Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";

// Simple instruction prompt to plan retirement
var puzzlePrompt = """
A farmer is on one side of a river with a wolf, a goat, and a cabbage. 
When he is crossing the river in a boat, he can only take one item with him at a time. 
The wolf will eat the goat if left alone together, and the goat will eat the cabbage if left alone together. 
How can the farmer transport the goat across the river without it being eaten?
""";

// Create a new chat history object using the new Chain of Thought system prompt
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPromptChainOfThought);
chatHistory.AddUserMessage(puzzlePrompt);

if (useAzureOpenAI)
{
    // Create a new Azure OpenAI Prompt Execution settings object
    #pragma warning disable SKEXP0010
    promptExecutionSettings = new AzureOpenAIPromptExecutionSettings { 
        SetNewMaxCompletionTokensEnabled = true,
        MaxTokens = 1500,
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0, 
        PresencePenalty = 0.0
        };
}
else
{
    // Create a new OpenAI Prompt Execution settings object
    promptExecutionSettings = new OpenAIPromptExecutionSettings { 
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0, 
        PresencePenalty = 0.0
        };
}

// Execute the chat completion request using the prompt and settings defined above
var decisionResponseChainOfThought = await chatCompletionService.GetChatMessageContentAsync(chatHistory, promptExecutionSettings);
// Display the result of the chat completion request as Markdown
decisionResponseChainOfThought.Content.DisplayAs("text/markdown");
("---").DisplayAs("text/markdown"); // Break of the ooutput section


| Step | Reasoning | Action |
|------|-----------|--------|
| 1 | The key constraint is avoiding situations where the wolf and goat are left together, or the goat and cabbage are left together when the farmer is absent. | Identify safe states and move items without violations. |
| 2 | If the farmer first takes the goat across, the wolf and cabbage left together are fine (no eating). | Take the goat across first and leave it on the far side. |
| 3 | The farmer returns alone to the original side to fetch another item. If he takes the wolf next, he must avoid leaving the wolf with goat. | Return alone. |
| 4 | Taking the wolf next would leave it with the goat, so instead, take the wolf but then bring the goat back to avoid conflict. | Take wolf across; bring goat back. |
| 5 | Now the goat is back, and the wolf is safe on the far side. The farmer can now take the cabbage across without danger (wolf and cabbage are fine together). | Take cabbage across and leave it with wolf. |
| 6 | Finally, return alone to fetch the goat, and bring goat across to far side where wolf and cabbage are already safely together. | Bring goat across. |
| 7 | Final arrangement: all three safely across with no conflict. | All done safely. |

**Sequence of Moves:**
1. Farmer takes Goat → Far Side  
2. Farmer returns alone → Near Side  
3. Farmer takes Wolf → Far Side  
4. Farmer brings Goat back → Near Side  
5. Farmer takes Cabbage → Far Side  
6. Farmer returns alone → Near Side  
7. Farmer takes Goat → Far Side  

Would you like me to also **draw a diagram** of each stage for clarity?

---

The Generative AI model has generated two different decision approaches to the same retirement decision question. An LLM can be used to decide which decision approach is of better quality. The evaluation could be done by another AI model or an AI model from different AI provider to reduce potential bias. 

After running the evaluation of the decision prompts below, notice the GenAI LLM model prefers the approach of the Chain of Thought (CoT) over the simple system prompt instruction. Furthermore, the AI model appreciates using the "Decision Communication" step with the applied Minto Pyramid.  

> 📝 **Note:** When executing the decision approach evaluation below, you may notice that sometimes Approach 1 is preferred. This happens as Approach 2 with Chain of Thought is considered verbose and not a clear as Approach 1 to the AI model.  In a real-world implementation, the prompt instruction will have this made very clear on what the criteria are. I specifically left that instruction out of the prompt to illustrate that even an AI model can vary it's preference!  

In [25]:
var systemPromptEvaluateResponses = """
You are an assistant that is evaluating an approach response to a question.
You will be provided with an important decision as well as two proposed approaches.
The two approaches are labeled "Approach 1" and "Approach 2".

Compare the two approaches and evaluate them based on their: 
Logical structured reasoning of approach, detail of the approach to the decision and the quality the communication. 

Create a final score between 1 and 10 for each approach based on the evaluation criteria.

Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";

var decisionEvaluationTemplateApproaches = $"""
Decision Scenario: 
{puzzlePrompt}
------------------------------------------------
Approach 1: 
{decisionResponse.Content} 
End of Approach 1
------------------------------------------------
Approach 2: 
{decisionResponseChainOfThought.Content} 
End of Approach 2
""";

// Create a new chat history object using the new Chain of Thought system prompt
var chatHistoryApproachEvaluation = new ChatHistory();
chatHistoryApproachEvaluation.AddSystemMessage(systemPromptEvaluateResponses);
chatHistoryApproachEvaluation.AddUserMessage(decisionEvaluationTemplateApproaches);

// Create a new OpenAI prompt execution settings object
if (useAzureOpenAI)
{
    // Create a new Azure OpenAI Prompt Execution settings object
    #pragma warning disable SKEXP0010
    promptExecutionSettings = new AzureOpenAIPromptExecutionSettings { 
        SetNewMaxCompletionTokensEnabled = true,
        MaxTokens = 1500,
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0, 
        PresencePenalty = 0.0
        };
}
else
{
    // Create a new OpenAI Prompt Execution settings object
    promptExecutionSettings = new OpenAIPromptExecutionSettings { 
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0, 
        PresencePenalty = 0.0
        };
}

// Execute the chat completion request using the prompt and settings defined above
var evaluationResponseApproach1 = await chatCompletionService.GetChatMessageContentAsync(chatHistoryApproachEvaluation, promptExecutionSettings);
// Display the result of the chat completion request as Markdown
"### Decision Approaches Evaluation: ".DisplayAs("text/markdown");
evaluationResponseApproach1.Content.DisplayAs("text/markdown");
("---").DisplayAs("text/markdown"); // Break of the output section

### Decision Approaches Evaluation: 

| Approach | Logical Structured Reasoning | Detail of Approach to Decision | Quality of Communication | Score |
|----------|------------------------------|--------------------------------|--------------------------|-------|
| Approach 1 | Presents a correct and sequential solution in a clear tabular form, but lacks upfront reasoning or explanation of why moves are chosen; logic is implied by the steps but not explicitly described. | Details all item locations for each step, which helps verify correctness; however, does not state rationale for each action or the constraints being addressed. | Table is concise and clear, but absence of reasoning limits accessibility for those unfamiliar with the puzzle logic. | 7 |
| Approach 2 | Offers explicit logical reasoning and constraints before and during each step, making the thought process transparent; each choice is justified to avoid conflicts. | Provides full sequence of moves and explains the reasoning behind them, ensuring constraints are always respected; more comprehensive than Approach 1. | Communicates clearly with both a reasoning column and a summarized sequence of moves; verbose but accessible and educational; even offers optional diagrams for further clarity. | 9 |

---

### Step 3 - Understanding Collective Intelligence for Decision Intelligence 

#### Collective Intelligence - Pooling Wisdom of Multiple Opinions 

> 📜 **_"The four conditions that characterize wise crowds: diversity of opinion, independence, decentralization, and aggregation."_**
>
> -- <cite>James Surowiecki (American journalist, author of The Wisdom of Crowds, and former columnist for The New Yorker)</cite> 

Imagine you’ve hurt your leg playing a long season of football games. You have another football season quickly approaching and you would like to understand how long will it take to heal properly. Additionally, you would like to understand the optimal medical treatment to ensure you are ready at the start of the next football season. You decide to visit three different doctors for their medical opinions. You don’t want to rely on just one doctor because each professional might notice something the others miss. Each medical specialist may decide to recommend a different approach based on their expertise or their own experience treating leg injuries. After you receive opinions from three different specialists, you think about their recommendations and then formulate a path forward (decision) for a treatment plan for your leg. The final treatment plan could be a result a variety of factors. For example, it could be simple and all three doctors could recommend 4-6 weeks rest. Clearly in that case there is a consensus with three different doctors on the treatment approach. What if the doctors diverge in their opinions? What if 2 of 3 doctors recommend rest and a third recommends an additional procedure on top of rest? What if all three doctors opinions totally diverge? Now you have to personally judge those doctor opinions collectively or potentially weight one doctor's opinion more significantly than others. This scenario illustrates a simplified version of collective intelligence, where pooling diverse expert opinions often leads to a more informed and balanced decision.

The idea of multiple experts arriving at a cohesive conclusion isn’t new. It has been demonstated that pooling the wisdom of multiple “opinions” can often outperform a single prediction. Those opinions don't have to be human experts. The opinions can be outcomes from a statistical model, a large survey, a machine learning model or Generative AI. You have probably have heard the terms: Wisdom of the Crowds, Collective Intelligence, Bootstrapping (technique in statistical analysis), Ensembling (Machine Learing) or Mixture of Experts (Generative AI). These are all similar techniques (with their own unique implementations) that derive of the core concept of "pooling wisdom of multiple opinions".  

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-SelfConsistency.png">

#### Examples of "Wisdom of the Crowds" at Scale

"Wisdom of the Crowds" can scale well beyond just a few doctor's opinions. In the examples below, note the number of samples collected for each situation is much greater than just a few doctors. In fact, the amount of samples collected can be in the hundreds and the Collective Intelligence power can hold true. 

**Francis Galton’s Ox-Weighing Experiment (1906)** 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-CollectiveIntelligence-OxWeight.png"> 

Sir Francis Galton, an English statistician, collected 787 guesses from fairgoers trying to estimate the weight of an ox. The average guess was 1,197 pounds, and the actual weight of the ox turned out to be 1,198 pounds. To Galton’s surprise, the average of these guesses (1,197 pounds) came remarkably close to the ox’s actual weight (1,198 pounds)—off by a single pound, or less than 0.1%. That level of accuracy was notable because the crowd was composed of a mixed audience: farmers with relevant experience, but also onlookers, tradespeople, and curious fairgoers with no particular expertise.

**Modern “Jelly Bean Jar” Contests**

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-CollectiveIntelligence-JellyBeans.png">

Many schools and charities run fundraisers where people pay to guess the number of jelly beans in a jar. It’s commonly observed that while individual guesses can be wildly off, the average of a sufficiently large group is typically within a small percentage (often under 5% error) of the true count. As an example, Michael Mauboussin ran an 2007 experiment where Mauboussin presented a jar of jelly beans to 73 Columbia Business School students. The students' guesses ranged from 250 to 4,100, with an average error of 62%. However, the group's average guess was 1,151, which represented only a 3% off the correct number. Only two students guessed better!

The cool part of the “Jelly Bean Jar” experiment is that it is very approachable to run yourself. In fact, you can do this with Generative AI as well. 

A short (5 minute) YouTube video illustrates running this experiment with human opinions:  
[![Jelly Bean Jar Experiment](https://img.youtube.com/vi/AuQdoAa2FUs/0.jpg)](https://www.youtube.com/watch?v=AuQdoAa2FUs)


**Netflix Recommendation Algorithm Million Dollar Prize**

In 2006, Netflix launched the million-dollar Netflix Prize competition, challenging participants to improve its movie recommendation algorithm (Cinematch) by at least 10 percent. Over three years, data scientists and researchers worldwide experimented with innovative approaches, culminating in 2009 when an ensemble team calling itself “BellKor’s Pragmatic Chaos” finally met the 10-percent threshold with 10.06%. This group had blended multiple models into a single “ensemble” algorithm that outperformed any single predictive approach. This demonstrated how synthesis of different predictions can be more powerful than one model alone. Although Netflix ultimately chose not to deploy the winning solution due to technical and privacy considerations, the competition is still regarded as a landmark moment in machine learning and collective problem-solving.  

More information can be found here: https://en.wikipedia.org/wiki/Netflix_Prize 

**“Ask the Audience” on Who Wants to Be a Millionaire?**

On the TV quiz show, contestants can use the “Ask the Audience” lifeline to poll the studio audience for the correct answer. Historical data shows that the audience collectively identifies the correct answer around 90% of the time, which is significantly more accurate than individual expert panels, or even the “Phone a Friend” lifeline.

**The U.S. Navy’s Hunt for the Missing Submarine Scorpion (1968)**

When the USS Scorpion, a nuclear-powered submarine, vanished in 1968, the Navy enlisted a broad range of specialists to harness their varied expertise and wisdom for a methodical search. By applying Bayesian statistical techniques to each person’s estimate of where the submarine might lie, they were able to synthesize these disparate perspectives into a single, remarkably accurate prediction. The team’s calculated guess put the submarine’s probable location just a few hundred yards from its actual resting place! This is a testament to the effectiveness of coalescing multiple expert viewpoints. This success with the USS Scorpion search later inspired more widespread use of Bayesian approaches for complex rescue and recovery operations, affirming the power of collective intelligence in high-stakes scenarios.

#### Statistics Explains why Collective Intelligence Works

It is important to understand that Collective Intelligence mechanisms are rooted in several statistics principles. This allows you to set up Collective Intelligence parameters appropriately for decision situations. Furthermore, this allows you to perform math (quantitative analysis) in the Decision Intelligence Execution step. The key statistical principles of Collective Intelligence:  
* **Average of Independent Predictions:** The average of many independent predictions, opinions or guesses will average itself out towards the true value. Each individual prediction can be assumed to have an error.  
Therefore the formula is:  Prediction = True Value + Error.  
As long as the errors are random, they will trend to cancel each other out on average. 
* **Law of Large Numbers:** The more guesses you have, the more the random errors balance out, reducing the overall variance of the collective guess. Mathematically, if you have a large number of independent predictions, the average guess converges on the True Value. 
* **Central Limit Theorem:** The distribution of preductions will tend toward a bell curve (a normal distribution) around the Predicted True Value. When you take the average (mean) of a large sample from a sample distribution like this, it will often fall near the true population distribution. This happens more often in real-life than statistics courses lead on.  

Statistics explaining Collective Intelligence (Wisdom of the Crowds) is greatly simplfied and there are things to consider when attempting to replicate this experiment with real-world decision situations:  

* **Bias Matters:** Collective Intelligence works best when individual biases do not all tilt in the same direction. For example, if you only poll basketball players about height-related predictions, it may introduce a huge bias towards larger height in a single direction. 

* **Independence:** The group’s diversity and independence of guesses are critical. A group of people trained in the same way or over-influenced by each other’s guesses might show correlated errors, undermining the benefit of aggregation.

* **Meaningful Sample Size:** The sampling error shrinks as you collect more predictions (guesses). Assume you have 𝑁 amount of independent predictions. If each prediction's random error in guessing has a standard deviation of 𝜎, then the standard error of the mean guess drops by about a factor of $\sqrt{𝑁}$. What does this mean? The more samples you add, the more reduction of the error you will notice. Furthermore, you will notice a larger reduction in error with the first initial predictions than further predictions made. Saying it another way. For example, if you have a certain error from 10 samples and you want to reduce that prediction error by half you need a total of 40 samples. Conversely, if you have 100 samples and you want to reduce that prediction error by half you need a total of 400 samples! 

### Step 4 - Implementing Collective Intelligence with Generative AI 

#### Collective Intelligence with AI Scenario Introduction

In this section, we will re-create the classic “guess the number of jelly beans” challenge, with a glass filled with Hershey’s Kisses. If you think about how these challenges are typically done, a human is presented a container (jar, glass, clear bucket) filled with candy. The human making the guess can look inside the container, inspect it, sometimes even hold the container and their visual senses can give it a sense of scale. 

In order to simulate this as best as possible for Artifical Intelligence (limited to only visual senses):
* Multiple images will be used (5 in total) of the same glass filled with Hershey's kisses
* The glass will be clear, so the main perspective of the contents are not obstructed 
* Each image will show a different perspective view of the glass containing the candy
* Each image will include two key reference points a soda can and a glass bottle, which we can assume the AI knows via it's training data sets
* Each image will include a single Hershey's Kisses as a scale reference point

The 5 images are shown below. Before scrolling down, can you guess the number of Hershey's Kisses?

<img style="display: block; margin: auto;" width ="200px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses1.jpg">

<img style="display: block; margin: auto;" width ="200px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses2.jpg">

<img style="display: block; margin: auto;" width ="200px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses3.jpg">

<img style="display: block; margin: auto;" width ="200px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses4.jpg">

<img style="display: block; margin: auto;" width ="200px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses5.jpg">

> 📝 Note: There are a total of 50 Hershey's Kisses in the glass. 

#### Single Point Estimates with Artificial Intelligence  

In order to implement Collective Intelligence with Generative AI, the first step is to be able to perform a single estimate. The code section below will use AI to perform a single point estimate based off of a single image of the glass filled with Hershey's kisses.

The code should be familiar by now. The main change is that we are adding a reference to the actual Hershey's Kisses image. This will provide the "eyes" to the Artificial Intelligence model.  

In [26]:
// Set the system prompt to behave like a decision intelligence assistant (persona)
var systemDecisionPrompt = """
You are an expert visual estimator.
Provide structured, logical, and comprehensive advice.

Output Format Instructions:
ONLY return a single integer as the answer. 
Do not include any other text, reasoning or explanation.

Example Expected Outputs (each line is a separate output):
12
100
1234
5443
45
""";

// Simple instruction prompt to plan retirement
var collectiveIntelligencePrompt = """
You have an image of a glass with Hershey's Kisses chocolates.
There is a soda can, soda bottle and a single hershey kiss outside provided for scale.

Estimate the number of Hershey's Kisses in the glass. 
Think through the estimate clearly using all the available information in the image.

Provide only a single number as the answer.
""";

// Uri of a single image of a Hershey's Kisses in a glass
Uri githubImageUri1 = new(
    "https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses1.jpg");

// Build Chat History with system prompt and the puzzle prompt
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemDecisionPrompt);
chatHistory.Add(
    new() {
        Role = AuthorRole.User,
        Items = [
            new TextContent { Text = collectiveIntelligencePrompt },
            new ImageContent { Uri = githubImageUri1 }
        ]
    });


// Create a new OpenAI prompt execution settings object
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
// Create a new OpenAI prompt execution settings object
if (useAzureOpenAI)
{
    // Create a new Azure OpenAI Prompt Execution settings object
    #pragma warning disable SKEXP0010
    promptExecutionSettings = new AzureOpenAIPromptExecutionSettings { 
        SetNewMaxCompletionTokensEnabled = true,
        MaxTokens = 5000,
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0
        };
}
else
{
    // Create a new OpenAI Prompt Execution settings object
    promptExecutionSettings = new OpenAIPromptExecutionSettings { 
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0
        };
}

// Execute the chat completion request using the prompt and settings defined above
var decisionResponse = await chatCompletionService.GetChatMessageContentAsync(chatHistory, promptExecutionSettings);
// Display the result of the chat completion request as Markdown
("AI Estimate of Hershey's Kisses in the glass - Using a Single Image: ").DisplayAs("text/markdown");
decisionResponse.Content.DisplayAs("text/markdown");
("---").DisplayAs("text/markdown"); // Break of the ooutput section

AI Estimate of Hershey's Kisses in the glass - Using a Single Image: 

36

---

A single image only provides a small section of context to the AI system. It will would be more powerful to provide several different images to provide various perspectives angles. The idea is that the AI system can use these images to perform a better estimate.

In [27]:
// Set the system prompt to behave like a decision intelligence assistant (persona)
var systemDecisionPrompt = """
You are an expert visual estimator.
Provide structured, logical, and comprehensive advice.

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
""";

// Simple instruction prompt to plan retirement
var collectiveIntelligencePromptMultipleImages = """
Estimate how many standard size Hershey's Kisses are inside a clear glass shown in five images.

Reference objects for scale (present in every photo)
- A 12 oz (355 ml) soda can
- A 8 oz (237 ml) soda bottle
- A single wrapped Hershey's Kiss placed outside the glass

Estimate the number of Hershey's Kisses in the glass. 
Think through the estimate clearly using all 5 of the available information in all of the images.

Provide only a single number as the answer.
""";

// Uris of a 5 images of a Hershey's Kisses in a glass
Uri githubImageUri1 = new(
    "https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses1.jpg");
Uri githubImageUri2 = new(
    "https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses2.jpg");
Uri githubImageUri3 = new(
    "https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses3.jpg");
Uri githubImageUri4 = new(
    "https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses4.jpg");
Uri githubImageUri5 = new(
    "https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses5.jpg");


// Build Chat History with system prompt and the puzzle prompt
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemDecisionPrompt);
chatHistory.Add(
    new() {
        Role = AuthorRole.User,
        Items = [
            // Provide 5 different images of the same glass with Hershey's Kisses chocolates
            new TextContent { Text = collectiveIntelligencePromptMultipleImages },
            new ImageContent { Uri = githubImageUri1 },
            new ImageContent { Uri = githubImageUri2 },
            new ImageContent { Uri = githubImageUri3 },
            new ImageContent { Uri = githubImageUri4 },
            new ImageContent { Uri = githubImageUri5 }
        ]
    });


// Create a new OpenAI prompt execution settings object
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
if (useAzureOpenAI)
{
    // Create a new Azure OpenAI Prompt Execution settings object
    #pragma warning disable SKEXP0010
    promptExecutionSettings = new AzureOpenAIPromptExecutionSettings { 
        SetNewMaxCompletionTokensEnabled = true,
        MaxTokens = 3500,
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0
        };
}
else
{
    // Create a new OpenAI Prompt Execution settings object
    promptExecutionSettings = new OpenAIPromptExecutionSettings { 
        // Uncomment if using a model that supports temperature, GPT-5 models do not support temperature (other than GPT-5-Chat)
        // Temperature = 0.3,
        TopP = 1.0, 
        FrequencyPenalty = 0.0
        };
}

// Execute the chat completion request using the prompt and settings defined above
var decisionResponse = await chatCompletionService.GetChatMessageContentAsync(chatHistory, promptExecutionSettings);
// Display the result of the chat completion request as Markdown 
("AI Estimate of Hershey's Kisses in the glass - Using 5 images: ").DisplayAs("text/markdown");
decisionResponse.Content.DisplayAs("text/markdown");
("---").DisplayAs("text/markdown"); // Break of the ooutput section

AI Estimate of Hershey's Kisses in the glass - Using 5 images: 

46

---

#### Adding variance to simulate multiple independent estimates with Artifical Intelligence



Re-running the same AI configuration over a single image will not simulate different independent perspectives. Simulating different perspectives can be challenging, but it can be done by combining various configurations and images. Variance will be added to this Collective Intelligence scenario by "crafting" an AI system by randomly selecting these options:   
* Two different AI models (GPT-4.1 and o4-mini) a "General AI" model and a "Reasoning AI" model
* Three different instruction prompts (Simple, Detailed, Chain of Thought)
* General AI Only: Three different Temeperature settings
* Reasoning AI Only: Three different Reasoning Effort Settings
* Using 3 random images from the 5 provided (10 different combinations)

General AI and Reasoning AI support different prompt settings. The main way to add variance to a General OpenAI model is to use the temperature setting. Conversely, for a reasoning model the primary manner to add variance to a Reasoning OpenAI model is with reasoning effort. Therefore, depending on which AI model has been selected it will either vary temperature (Genral AI) or reasoning effort (Reasoning AI) not both. 

Each type of run (Reasoning AI or General AI) has: 3 * 3 * 10 = 90 varied ways it can be put together. Because, ther are two types of paths (Reasoning AI and General AI), there are a total of 180 (2 * 90) different combinations for this "AI System". That is quite a good amount of variance that can be introduced to estimate the amount of Hershey's Kisses in the glass.  

In [28]:
using System.Net.Http;

var httpClientAI = new HttpClient();
httpClientAI.Timeout = TimeSpan.FromSeconds(300);

// Example to build a Kernel with three different AI services
var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        modelId: azureOpenAIModelDeploymentName,
        deploymentName: azureOpenAIModelDeploymentName,
        endpoint: azureOpenAIEndpoint,
        apiKey: azureOpenAIAPIKey,
        httpClient: httpClientAI,
        serviceId: "GeneralAI")
    .AddAzureOpenAIChatCompletion(
        modelId: azureOpenAIReasoningModelDeploymentName,
        deploymentName: azureOpenAIReasoningModelDeploymentName,
        endpoint: azureOpenAIReasoningEndpoint,
        apiKey: azureOpenAIReasoningAPIKey,
        httpClient: httpClientAI,
        serviceId: "ReasoningAI")
    .Build();

// Retrieve the chat completion service for each respective AI service (GeneralAI and ReasoningAI)
var chatCompletionServiceGeneralAI = semanticKernel.GetRequiredService<IChatCompletionService>("GeneralAI");
var chatCompletionServiceReasoningAI = semanticKernel.GetRequiredService<IChatCompletionService>("ReasoningAI");

In [29]:
// Set the system prompt to behave like a decision intelligence assistant (persona)
var systemDecisionPrompt = """
You are an expert visual estimator.
Provide structured, logical, and comprehensive advice.

You are provided with several photographs of the **same** glass of Hershey's Kisses chocolates

Reference objects for scale (present in every photo image)
- A 12 oz soda can
- A 8 oz soda bottle
- A single wrapped Hershey's Kiss placed outside the glass

Estimate the number of Hershey's Kisses in the glass. 
Think through the estimate clearly using all the available information in all of the images.

Output Format Instructions:
ONLY return a single integer as the answer. 
Do not include any other text, reasoning or explanation.

Example Expected Outputs (each line is a separate output):
12
56
1234
5443
30
""";

// Define Three different prompts for the same task
var collectiveIntelligencePromptMultipleImagesSimple = """
Estimate how many Hershey's Kisses are in this glass from the images.
""";

var collectiveIntelligencePromptMultipleImagesDetailed = """
Examine perspective cues (external objects, table height, glass) and reason about the glass 
geometry (cylinder? rectangular prism?) to estimate volume and capacity.
Estimate the number of Hershey's Kisses in the glass.

Think clearly, but *only* return a single number.
""";

var collectiveIntelligencePromptMultipleImagesChainOfThought = """
First, think step-by-step about:
1. Glass dimensions (height, diameter)
2. Count number of Hershey's Kisses vertically from a single column
3. Count number of Hershey's Kisses horizontally from a single row
3. Expected total count: total number vertically * (2 * total number horizontally)

Then, provide your final estimate of the number of Hershey's Kisses in the glass.

Think silently, then give **one integer** answer on the last line.
""";

string[] collectiveIntelligencePrompts = new[]
{
    collectiveIntelligencePromptMultipleImagesSimple,
    collectiveIntelligencePromptMultipleImagesDetailed,
    collectiveIntelligencePromptMultipleImagesChainOfThought
};

// Define the different URI images of the glass with Hershey's Kisses chocolates
Uri[] allImageUris =
{
    new Uri("https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses1.jpg"),
    new Uri("https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses2.jpg"),
    new Uri("https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses3.jpg"),
    new Uri("https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses4.jpg"),
    new Uri("https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses5.jpg")
};
// Define three different temperature settings for the General AI service
float[] topProbabilities = { 0.1f, 0.5f, 1.0f };
// Define three different reasoning efforts for the Reasoning AI service
string[] reasoningEfforts = { "low", "medium", "high" };
// Define two different AI services
string[] aiServices = { "GeneralAI", "ReasoningAI" };

var aiEstimates = new List<int>();
var numberOfIterations = 10; 
for (int i = 0; i != numberOfIterations; i++)
{
    Console.WriteLine($"Iteration {i + 1} of {numberOfIterations}: ");

    // Randomize the choice of collective intelligence prompt, AI service, temperature, reasoning effort, and selection of 3 images
    var rng = new Random();
    var collectiveIntelligencePromptInteger = rng.Next(collectiveIntelligencePrompts.Length);
    string collectiveIntelligencePromptChosen     = collectiveIntelligencePrompts[collectiveIntelligencePromptInteger];
    string aiServiceChosen     = aiServices[rng.Next(aiServices.Length)];
    float  topPChosen       = topProbabilities[rng.Next(topProbabilities.Length)];
    string reasoningChosen  = reasoningEfforts[rng.Next(reasoningEfforts.Length)];
    var imagesChosen = allImageUris
        .OrderBy(_ => rng.Next())
        .Take(3)
        .Select(uri => new ImageContent { Uri = uri })
        .ToList();

    Console.WriteLine($"Collective Intelligence Prompt Number Chosen: {collectiveIntelligencePromptInteger+1}");
    Console.WriteLine($"AI Service Chosen: {aiServiceChosen}");
    if (aiServiceChosen == "ReasoningAI")
    {
        Console.WriteLine($"Reasoning Effort Chosen: {reasoningChosen}");
    }
    else
    {
        Console.WriteLine($"Top Chosen: {topPChosen}");
    }
    Console.WriteLine($"Images Chosen: {string.Join(", ", imagesChosen.Select(i => i.Uri))}");

    // GeneralAI and ReasoningAI will have different PromptExecutionSettings
    var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings();

    if (aiServiceChosen == "GeneralAI")
    {
        openAIPromptExecutionSettings.TopP = topPChosen;
    }
    else if (aiServiceChosen == "ReasoningAI")
    {
        openAIPromptExecutionSettings.ReasoningEffort = reasoningChosen;
    }

    // Build Chat History with system prompt and the puzzle prompt
    var chatHistory = new ChatHistory();
    chatHistory.AddSystemMessage(systemDecisionPrompt);
    chatHistory.Add(
        new() {
            Role = AuthorRole.User,
            Items = [
                new TextContent { Text = collectiveIntelligencePromptChosen },
                new ImageContent { Uri = allImageUris[0] },
                new ImageContent { Uri = allImageUris[1] },
                new ImageContent { Uri = allImageUris[2] }
            ]
        });

    var chatCompletionServiceToUse = semanticKernel.GetRequiredService<IChatCompletionService>(aiServiceChosen);
    // Execute the chat completion request using the prompt and settings defined above
    var decisionResponse = await chatCompletionServiceToUse.GetChatMessageContentAsync(chatHistory, openAIPromptExecutionSettings);
    // Display the result of the chat completion request as Markdown
    //decisionResponse.Content.DisplayAs("text/markdown");


    Console.WriteLine($"Raw response: {decisionResponse.Content.ToString()}");
    var estimate = 0;
    if (int.TryParse(decisionResponse.Content.ToString(), out estimate))
    {
        Console.WriteLine($"Estimate: {estimate}");
        aiEstimates.Add(estimate);
    }
    else
    {
        Console.WriteLine("Failed to parse the estimate from the response.");
    }
    ("---").DisplayAs("text/markdown"); // Break of the ooutput section
}

// Display the average estimate from all AI services
if (aiEstimates.Count > 0)  
{
    var averageEstimate = aiEstimates.Average();
    Console.WriteLine($"Mininum estimate of Hershey's Kisses in glass: {aiEstimates.Min()}");
    Console.WriteLine($"Maximum estimate of Hershey's Kisses in glass: {aiEstimates.Max()}");
    Console.WriteLine($"Average estimate of Hershey's Kisses in glass: {averageEstimate}");
    Console.WriteLine($"Actual number of Hershey's Kisses in glass: {50}");
}
else
{
    Console.WriteLine("No estimates were generated.");
}
("---").DisplayAs("text/markdown"); // Break of the ooutput section

Iteration 1 of 10: 
Collective Intelligence Prompt Number Chosen: 1
AI Service Chosen: ReasoningAI
Reasoning Effort Chosen: high
Images Chosen: https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses3.jpg, https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses1.jpg, https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses5.jpg
Raw response: 40
Estimate: 40


---

Iteration 2 of 10: 
Collective Intelligence Prompt Number Chosen: 2
AI Service Chosen: ReasoningAI
Reasoning Effort Chosen: medium
Images Chosen: https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses2.jpg, https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses5.jpg, https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/refs/heads/main/Images/CodeExercises/CollectiveIntelligence/HersheyKisses3.jpg


Error: Command cancelled.

What does the analysis show from the simulation of 10 runs above? The range is quite dramatic, with the lowest estimate on record 34 and the highest being 120. The average estimate is 56.9, which is pretty close to the actual count of 50 Hershey's Kisses in the glass. This translates to an average error of about 7 Hershey's Kisses or ~14%. Running the simulations at a much greater number or increasing the variance should lower that number even further.

Another way to interpet the Collective Intelligence simulation is to look at individual results. For example, lets assume the "AI System" was not created and just a single run was performed. How well should you trust these results? Running the code below illustrates the difference between each individual AI estimate and the actual number of Hershey's Kisses in the glass. What should be facinating is that 9 times out of 10 an individual AI estimate has a higher difference than the "collective average" of the simulations. Therefore, in this example the Collective Intelligence of 10 independent AI estimates of Hershey's Kisses 90% of the time will provide a better estimate compared to a single AI estimate. 

In [ ]:
// Illustrate the difference between individual estimates and the actual number of Hershey's Kisses
foreach (var estimate in aiEstimates)
{
    Console.WriteLine($"AI Estimate: {estimate}. Differenmce from actual: {Math.Abs(estimate - 50)}");
}

// Count of esimtates that are greater than the average distance from the actual number of Hershey's Kisses
var countGreaterThanAverage = aiEstimates.Count(e => Math.Abs(e - 50) > Math.Abs(aiEstimates.Average() - 50));
Console.WriteLine($"Count of estimates greater than average distance from actual: {countGreaterThanAverage} from {aiEstimates.Count}");

AI Estimate: 36. Differenmce from actual: 14
AI Estimate: 95. Differenmce from actual: 45
AI Estimate: 34. Differenmce from actual: 16
AI Estimate: 120. Differenmce from actual: 70
AI Estimate: 56. Differenmce from actual: 6
AI Estimate: 38. Differenmce from actual: 12
AI Estimate: 38. Differenmce from actual: 12
AI Estimate: 37. Differenmce from actual: 13
AI Estimate: 35. Differenmce from actual: 15
AI Estimate: 80. Differenmce from actual: 30
Count of estimates greater than average distance from actual: 9 from 10


This example was very simple, but it showed that the "Wisdom of the Crowds" (Collective Intelligence) can be applied to Generative AI to improve estimates, reducing uncertainty and potentially providing better forecasts. This direct concept can be applied more advanced decisions simply by creating simulations that introduce proper variance in a complete AI system. 

Imagine that this system wasn't predicting the amount of Hershey's Kisses in a glass. Rather image the AI system was predicting sales of widgets in millions of dollars. Having an estimate between 36 million or 110 million has huge variance and implications. AI systems that do not have perfect information (can't see all of the Hershey's Kisses to just count them) will provide quite varied point estimates that have minimal value. Collective Intelligence techniques demonstrate the ability to converge to a "true" estimate range. This module proved that these techniques clearly apply to AI systems.